In [1]:
import csv
import numpy as np
import collections
import scipy.linalg as linalg
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
import notebookutil as nbu
import sys
sys.meta_path.append(nbu.NotebookFinder())

In [19]:
delta_lambda = 10
xaxis = range(380, 731, delta_lambda)
output_flag = True

In [5]:
def Fit_1931(wave):
    #%
    #% xFit_1931
    #% Approximation of X component.
    #%
    
    t1 = wave-442.0
    if(wave < 442.0):
        t1 = t1 * 0.0624
    else:
        t1 = t1 * 0.0374
    t2 = wave-599.8
    if(wave < 599.8):
        t2 = t2 * 0.0264
    else:
        t2 = t2 * 0.0323
    t3 = wave-501.1
    if(wave < 501.1):
        t3 = t3 * 0.0490
    else:
        t3 = t3 * 0.0382
    approxX = 0.362*math.exp(-0.5*t1*t1) + 1.056*math.exp(-0.5*t2*t2) - 0.065*math.exp(-0.5*t3*t3)
    
    #%
    #% yFit_1931
    #% Approximation of Y component
    #%
               
    t1 = wave-568.8  
    if (wave < 568.8):
        t1 = t1 * 0.0213
    else:
        t1 = t1 * 0.0247     
    t2 = wave-530.9
    if (wave < 530.9):
        t2 = t2 * 0.0613
    else:
        t2 = t2 * 0.0322 
    approxY = 0.821*math.exp(-0.5*t1*t1) + 0.286*math.exp(-0.5*t2*t2)
  
    #%
    #% zFit_1931
    #% Approximation of Z component.
    #%
            
    t1 = wave-437.0
    if( wave < 437.0):
        t1 = t1 * 0.0845
    else:
        t1 = t1 * 0.0278     
    t2 = wave-459.0
    if (wave < 459.0):
        t2 = t2 * 0.0385
    else:
        t2 = t2 * 0.0725
    approxZ = 1.217*math.exp(-0.5*t1*t1) + 0.681*math.exp(-0.5*t2*t2)
    return np.array([approxX, approxY, approxZ])

In [6]:
def make_xyz1931_10nm(xaxis):
    fit_xyz1931 = []
    for r in xaxis:
        fit_xyz1931.append(Fit_1931(r))
    return np.array(fit_xyz1931).T

In [7]:
#matrixs　XYZ to RGB
def make_xyz_to_rgb_matrix():
    rx = 0.64
    ry = 0.33   #%data from W3C http://www.w3.org/Graphics/Color/sRGB 
    gx = 0.3
    gy = 0.6
    bx = 0.15
    by = 0.06
    wx = 0.3333
    wy = 0.3333 #% equal energy white        
    wY = 106.8  #%should be the area under the XYZ curves, picked to make white = 1
    Yovery = wY / wy
    D =  (rx * (gy - by) + gx * (by - ry) + bx * (ry - gy))
    Cr = (Yovery  * (wx * (gy - by) - wy * (gx - bx) + gx * by - bx * gy) / D)
    Cg = (Yovery  * (wx * (by - ry) - wy * (bx - rx) + bx * ry - rx * by) / D)
    Cb = (Yovery  * (wx * (ry - gy) - wy * (rx - gx) + rx * gy - gx * ry) / D)
    toXYZ = np.array([[rx * Cr,  gx * Cg,   bx * Cb],
                      [ry * Cr,  gy * Cg,   by * Cb],
                      [(1.0 - (rx + ry)) * Cr,    (1.0 - (gx + gy)) * Cg,   (1.0 - (bx + by)) * Cb]])                 
    toRGB = np.linalg.inv(toXYZ)
    return toRGB

In [8]:
def real(a):
    return a.real

In [9]:
def load_principal_components(input_file_name, 
                              pcnum=3, header_flag=True, show_flag=False):
    components = []
    num = 1
    x = xaxis
    ifile = open(input_file_name,"r")
    idata = csv.reader(ifile)
    if(header_flag):
        idata.next()
    for line in idata:
        component = map(complex, line[1:])
        components.append(map(real, component))
        num = num + 1
        if(show_flag):
            plt.plot(x, map(real, component))
        if(num > pcnum):
            break
    ifile.close()
    if(show_flag):
        plt.show()
    return np.array(components).T

In [10]:
def load_spectra(filename, offset=1, is_header=True):
    spectra = []
    ifile = open(filename, "r")
    idata = csv.reader(ifile)
    if(is_header):
        idata.next()
    for line in idata:
        spectra.append(map(float, line[offset:]))
    ifile.close()
    return np.array(spectra)

In [14]:
#makeRGB 3 channels data from spectrum
def make_XYZ_from_spectrum(spectrum, xyzfunc=make_xyz1931_10nm(xaxis), bin_lambda=delta_lambda):
    return (bin_lambda*xyzfunc.dot(spectrum))

In [15]:
def make_RGB_from_spectrum(spectrum, xyz_to_rgb_matrix, 
                           xyzfunc=make_xyz1931_10nm(xaxis), bin_lambda=delta_lambda):
    return xyz_to_rgb_matrix.dot(bin_lambda*xyzfunc.dot(spectrum))

In [18]:
def write_result_with_header(res, filename, index, xaxis=xaxis):
    wfile = open(filename, "wb")
    writer = csv.writer(wfile)
    writer.writerow(["name"] + xaxis)
    for num, t in enumerate(res):
        writer.writerow([index[num]] + list(t))
    wfile.close()
    return 0

In [17]:
xyz_to_rgb_matrix = make_xyz_to_rgb_matrix()
xyz1931_10nm = make_xyz1931_10nm(xaxis)

In [1]:
def show_minimum_error(clusters_size):
    file_name = "EvaluateClustering{}.csv".format(clusters_size)
    print(file_name)
    ifile = open(file_name, "r")
    idata = csv.reader(ifile)
    idata.next()
    min_data = []
    min_err = 1
    for line in idata:
        if(float(line[1]) < min_err):
            min_data = line
            min_err = float(line[1])
    print(min_data)
    print("average error :", min_err)
    print("new tree :", min_data[5:])
    ifile.close()
    return min_data[5:]

In [ ]:
def reshape_new_tree_data(tree_data_str):
    new_tree_data = []
    for data in tree_data_str:
        reshaped_data = list(map(float, data[1:-1].split(",")))
        reshaped_data[1] = int(reshaped_data[1])
        reshaped_data[2] = int(reshaped_data[2])
        new_tree_data.append(reshaped_data)
    return new_tree_data